In [92]:
from typing import Set, Dict, Tuple, Union
from dataclasses import dataclass, field, asdict

In [67]:
SKILL = {"athletics": "strength",
          "acrobatics": "dexterity",
          "sleight_of_hand": "dexterity",
          "stealth": "dexterity",
          "arcana": "intelligence",
          "history": "intelligence",
          "investigation": "intelligence",
          "nature": "intelligence",
          "religion": "intelligence",
          "animal_handling": "wisdom",
          "insignt": "wisdom",
          "medicine": "wisdom",
          "perception": "wisdom",
          "survival": "wisdom",
          "deception": "charisma",
          "intimidation": "charisma",
          "performance": "charisma",
          "persuasion": "charisma"
         }

In [68]:
@dataclass
class Class():
    name: str
    level: int
        
    # disallows duplicate classes in a set
    def __hash__(self):
        return hash(self.name)

In [81]:
@dataclass
class Character():
    name: str
    race: str
    strength: int
    dexterity: int
    constitution: int
    intelligence: int
    wisdom: int
    charisma: int
    proficiencies: Set[str] = field(default_factory=set)
    skill_modifiers: Dict[str, Tuple[str, int]] = field(default_factory=dict)
    classes: Set[Class] = field(default_factory=set)
        
    def __eq__(self, other):
        if isinstance(other, Character):
            return (self.name == other.name)
        return False
        
    @property
    def level(self):
        total_level = sum(cls.level for cls in self.classes)
        return total_level or 1
    
    @property
    def proficiency(self):
        return (self.level + 3) // 4 + 1
    
    def add_proficiency(self, skill: str):
        self.proficiencies.add(skill)
        
    def add_class(self, cls: Class):
        if not isinstance(cls, Class):
            raise Exception('can only add Class objects')
        self.classes.add(cls)
    
    def get_modifier(self, skill_or_attribute: str):
        if (skill := getattr(self, skill_or_attribute, None)) is not None:
            return (skill - 10) // 2
        
        attr = SKILL.get(skill_or_attribute)
        base_mod = (getattr(self, attr) - 10) // 2
        
        prof_mod = 0 if skill_or_attribute not in self.proficiencies else self.proficiency
        
        return base_mod + prof_mod

In [82]:
thoros = Character("Thoros", "Human", 9, 12, 16, 20, 10, 9)


In [83]:
thoros.add_proficiency('arcana')

In [84]:
thoros.add_class(Class('wizard', 5))

In [64]:
thoros.get_modifier('arcana')

8

In [112]:
dolfur = Character("Dolfur", "Dwarf", 10, 10, 10, 10, 10, 10)

In [110]:
d = asdict(thoros)
d

{'name': 'Thoros',
 'race': 'Human',
 'strength': 9,
 'dexterity': 12,
 'constitution': 16,
 'intelligence': 20,
 'wisdom': 10,
 'charisma': 9,
 'proficiencies': {'arcana'},
 'skill_modifiers': {},
 'classes': {Class(name='wizard', level=5)}}

In [111]:
Character(**d)

Character(name='Thoros', race='Human', strength=9, dexterity=12, constitution=16, intelligence=20, wisdom=10, charisma=9, proficiencies={'arcana'}, skill_modifiers={}, classes={Class(name='wizard', level=5)})

In [156]:
Owner = str
CharacterName = str

@dataclass
class Rolodex():
    owners: Dict[CharacterName, Set[Owner]] = field(default_factory=dict)
    characters: Dict[CharacterName, Character] = field(default_factory=dict)
        
    def add_character(self, owners: Union[Owner, List[Owner]], character: Character):
        if character.name in self.characters:
            raise Exception("character is already in rolodex!")
        if len(owners) == 0:
            raise Exception("someone must own the character")
        if isinstance(owners, str):
            owners = [owners]
        self.characters[character.name.lower()] = character
        self.owners[character.name.lower()] = owners
        
    def get_character(self, user: str, character_name: str):
        character_name = character_name.lower()
        if character_name not in self.characters:
            raise Exception("there is no character by this name")
        if user not in self.owners[character_name]:
            raise Exception("you cannot access this character")
        return self.characters[character_name]
    
    def store(self):
        with open('characters.txt', 'w') as fout:
            fout.write(str(asdict(self)))
            
    @staticmethod
    def load():
        with open('characters.txt', 'r') as fin:
            rawdata = fin.read()
        data = eval(rawdata)
        return Rolodex(**data)

In [142]:
rd = Rolodex()

In [143]:
rd.add_character('FrenchyRaoul', thoros)

In [144]:
rd.add_character('FrenchyRaoul', dolfur)

In [145]:
rd.store()

In [157]:
Rolodex.load()

Rolodex(owners={'thoros': ['FrenchyRaoul'], 'dolfur': ['FrenchyRaoul']}, characters={'thoros': {'name': 'Thoros', 'race': 'Human', 'strength': 9, 'dexterity': 12, 'constitution': 16, 'intelligence': 20, 'wisdom': 10, 'charisma': 9, 'proficiencies': {'arcana'}, 'skill_modifiers': {}, 'classes': {Class(name='wizard', level=5)}}, 'dolfur': {'name': 'Dolfur', 'race': 'Dwarf', 'strength': 10, 'dexterity': 10, 'constitution': 10, 'intelligence': 10, 'wisdom': 10, 'charisma': 10, 'proficiencies': set(), 'skill_modifiers': {}, 'classes': set()}})

In [107]:
rd.get_character("FrenchyRaoul", "thoros")

Character(name='Thoros', race='Human', strength=9, dexterity=12, constitution=16, intelligence=20, wisdom=10, charisma=9, proficiencies={'arcana'}, skill_modifiers={}, classes={Class(name='wizard', level=5)})

In [109]:
import dataclasses

In [120]:
str(asdict(rd))

"{'owners': {'thoros': ['FrenchyRaoul'], 'dolfur': ['FrenchyRaoul']}, 'characters': {'thoros': {'name': 'Thoros', 'race': 'Human', 'strength': 9, 'dexterity': 12, 'constitution': 16, 'intelligence': 20, 'wisdom': 10, 'charisma': 9, 'proficiencies': {'arcana'}, 'skill_modifiers': {}, 'classes': {Class(name='wizard', level=5)}}, 'dolfur': {'name': 'Dolfur', 'race': 'Dwarf', 'strength': 10, 'dexterity': 10, 'constitution': 10, 'intelligence': 10, 'wisdom': 10, 'charisma': 10, 'proficiencies': set(), 'skill_modifiers': {}, 'classes': set()}}}"